# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [32]:
# import libraries
import sqlite3
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk import pos_tag
import re

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///etl_pipeline.db')
df = pd.read_sql("SELECT * FROM disaster_response", engine)
X = df['message']
Y = df.drop(['message', 'genre', 'id', 'original'], axis=1)
category_names = Y.columns.tolist()

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    tokens = word_tokenize(text)   
    lemmatizer = WordNetLemmatizer()
    
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stopwords.words("english")]    
    
    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y,test_size=0.30,random_state=42)

In [6]:
pipeline.fit(X_train, Y_train)


Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
from sklearn.metrics import classification_report
Y_pred=pipeline.predict(X_test)     

In [8]:
Y_test.values

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 1, 1, 0],
       ..., 
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [16]:
for i, col in enumerate(Y_test):
    print(col,':')
    print(classification_report(Y_test.iloc[:,i], Y_pred[:,i]))
    print('--------------------------------')

related :
             precision    recall  f1-score   support

          0       0.61      0.45      0.52      1873
          1       0.84      0.91      0.87      5934
          2       0.32      0.29      0.31        58

avg / total       0.78      0.79      0.78      7865

--------------------------------
request :
             precision    recall  f1-score   support

          0       0.90      0.97      0.93      6533
          1       0.78      0.45      0.57      1332

avg / total       0.88      0.89      0.87      7865

--------------------------------
offer :
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      7829
          1       0.00      0.00      0.00        36

avg / total       0.99      1.00      0.99      7865

--------------------------------
aid_related :
             precision    recall  f1-score   support

          0       0.76      0.84      0.80      4646
          1       0.73      0.62      0.67      3219



/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [23]:
parameters = {
    'clf__estimator__n_estimators': [40, 60, 80],
    'clf__estimator__min_samples_split': [4, 6, 8],    
}

cv = GridSearchCV(pipeline, param_grid=parameters, n_jobs=5, verbose=2)

In [24]:
cv.fit(X_train, Y_train)

Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV] clf__estimator__min_samples_split=4, clf__estimator__n_estimators=40 
[CV] clf__estimator__min_samples_split=4, clf__estimator__n_estimators=40 
[CV] clf__estimator__min_samples_split=4, clf__estimator__n_estimators=40 
[CV] clf__estimator__min_samples_split=4, clf__estimator__n_estimators=60 
[CV] clf__estimator__min_samples_split=4, clf__estimator__n_estimators=60 
[CV]  clf__estimator__min_samples_split=4, clf__estimator__n_estimators=40, total=17.9min
[CV] clf__estimator__min_samples_split=4, clf__estimator__n_estimators=60 
[CV]  clf__estimator__min_samples_split=4, clf__estimator__n_estimators=40, total=17.9min
[CV] clf__estimator__min_samples_split=4, clf__estimator__n_estimators=80 
[CV]  clf__estimator__min_samples_split=4, clf__estimator__n_estimators=40, total=18.0min
[CV] clf__estimator__min_samples_split=4, clf__estimator__n_estimators=80 
[CV]  clf__estimator__min_samples_split=4, clf__estimator__n_estimator

[Parallel(n_jobs=5)]: Done  27 out of  27 | elapsed: 154.6min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=5,
       param_grid={'clf__estimator__n_estimators': [40, 60, 80], 'clf__estimator__min_samples_split': [4, 6, 8]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=2)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [27]:
print(cv.best_params_)
best_model = cv.best_estimator_
print(best_model)

{'clf__estimator__min_samples_split': 4, 'clf__estimator__n_estimators': 60}
Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])


In [28]:
Y_pred_cv = best_model.predict(X_test)

In [31]:
from sklearn.metrics import accuracy_score, precision_score, recall_score

for i, col in enumerate(Y_test):
    print(col,':')
    print('Accuracy: ', accuracy_score(Y_test.iloc[:,i], Y_pred_cv[:,i]))
    print('Precision: ', precision_score(Y_test.iloc[:,i], Y_pred_cv[:,i], average='weighted'))
    print('Recall: ', recall_score(Y_test.iloc[:,i], Y_pred_cv[:,i], average='weighted'))
    print('--------------------------------')


related :
Accuracy:  0.808137317228
Precision:  0.793924207448
Recall:  0.808137317228
--------------------------------
request :
Accuracy:  0.898156389065
Precision:  0.892520009638
Recall:  0.898156389065
--------------------------------
offer :
Accuracy:  0.995422759059
Precision:  0.990866469253
Recall:  0.995422759059
--------------------------------
aid_related :
Accuracy:  0.772536554355
Precision:  0.771412342806
Recall:  0.772536554355
--------------------------------
medical_help :
Accuracy:  0.921169739352
Precision:  0.899325502359
Recall:  0.921169739352
--------------------------------
medical_products :
Accuracy:  0.950794659886
Precision:  0.943896474002
Recall:  0.950794659886
--------------------------------
search_and_rescue :
Accuracy:  0.976350921805
Precision:  0.970409308388
Recall:  0.976350921805
--------------------------------
security :
Accuracy:  0.981436745073
Precision:  0.9637127189
Recall:  0.981436745073
--------------------------------
military :
Accu

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


 0.942530197076
Precision:  0.938026261585
Recall:  0.942530197076
--------------------------------
fire :
Accuracy:  0.98944691672
Precision:  0.984404922537
Recall:  0.98944691672
--------------------------------
earthquake :
Accuracy:  0.971646535283
Precision:  0.970685154979
Recall:  0.971646535283
--------------------------------
cold :
Accuracy:  0.980165289256
Precision:  0.978510290025
Recall:  0.980165289256
--------------------------------
other_weather :
Accuracy:  0.947743165925
Precision:  0.933318891227
Recall:  0.947743165925
--------------------------------
direct_report :
Accuracy:  0.849968213605
Precision:  0.838647482829
Recall:  0.849968213605
--------------------------------


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [35]:
pipeline2 = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(AdaBoostClassifier()))
])

In [37]:
pipeline2.fit(X_train, Y_train)


Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))])

In [38]:
Y_pred2 = pipeline2.predict(X_test)     

In [39]:
for i, col in enumerate(Y_test):
    print(col,':')
    print(classification_report(Y_test.iloc[:,i], Y_pred2[:,i]))
    print('--------------------------------')

related :
             precision    recall  f1-score   support

          0       0.62      0.12      0.20      1873
          1       0.77      0.98      0.86      5934
          2       0.33      0.05      0.09        58

avg / total       0.73      0.77      0.70      7865

--------------------------------
request :
             precision    recall  f1-score   support

          0       0.90      0.97      0.93      6533
          1       0.75      0.49      0.60      1332

avg / total       0.88      0.89      0.88      7865

--------------------------------
offer :
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      7829
          1       0.00      0.00      0.00        36

avg / total       0.99      0.99      0.99      7865

--------------------------------
aid_related :
             precision    recall  f1-score   support

          0       0.76      0.86      0.81      4646
          1       0.75      0.60      0.67      3219



### 9. Export your model as a pickle file

In [40]:
import pickle
with open('model.pkl', 'wb') as pklfile:
    pickle.dump(best_model, pklfile)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.